In [1]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import numpy as np
import os


ERA5_NC = r"C:\Users\Ankit\Datasets_Forest_fire\compressed_cleaned_era5_2015_2016.nc"
OUT_DIR = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks"
os.makedirs(OUT_DIR, exist_ok=True)


ds = xr.open_dataset(ERA5_NC)

time_dim = "valid_time"
lat_dim = "latitude"
lon_dim = "longitude"

times = ds[time_dim].values
lat = ds[lat_dim].values
lon = ds[lon_dim].values


if lat[0] > lat[-1]:
    ds = ds.reindex({lat_dim: list(reversed(lat))})
    lat = ds[lat_dim].values


res_lon = abs(lon[1] - lon[0])
res_lat = abs(lat[1] - lat[0])
transform = from_origin(lon.min(), lat.max(), res_lon, res_lat)

for var in ds.data_vars:
    da = ds[var]  # DataArray
    print(f"Processing {var} with dims {da.dims}")

    
    arr = da.values.astype("float32")

    n_bands = arr.shape[0]
    profile = {
        "driver": "GTiff",
        "height": arr.shape[1],
        "width": arr.shape[2],
        "count": n_bands,
        "dtype": "float32",
        "crs": "EPSG:4326",
        "transform": transform,
        "tiled": True,            
        "compress": "deflate"    
    }

    out_tif = os.path.join(OUT_DIR, f"ERA5_{var}_2015_2016_stack.tif")

    with rasterio.open(out_tif, "w", **profile) as dst:
        dst.write(arr)  

    print(f"✅ Saved {var} to {out_tif} ({n_bands} time bands)")


Processing t2m with dims ('valid_time', 'latitude', 'longitude')
✅ Saved t2m to C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_t2m_2015_2016_stack.tif (17544 time bands)
Processing d2m with dims ('valid_time', 'latitude', 'longitude')
✅ Saved d2m to C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_d2m_2015_2016_stack.tif (17544 time bands)
Processing u10 with dims ('valid_time', 'latitude', 'longitude')
✅ Saved u10 to C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_u10_2015_2016_stack.tif (17544 time bands)
Processing v10 with dims ('valid_time', 'latitude', 'longitude')
✅ Saved v10 to C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_v10_2015_2016_stack.tif (17544 time bands)
Processing tp with dims ('valid_time', 'latitude', 'longitude')
✅ Saved tp to C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_tp_2015_2016_stack.tif (17544 time bands)


In [3]:
print("Dims:", da.dims)
print("Coords:", list(da.coords))


Dims: ('valid_time', 'latitude', 'longitude')
Coords: ['number', 'valid_time', 'latitude', 'longitude', 'expver']


In [2]:
import rasterio
import numpy as np

# List of ERA5 stacked files
era5_files = [
    r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_t2m_2015_2016_stack.tif",
    r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_d2m_2015_2016_stack.tif",
    r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_u10_2015_2016_stack.tif",
    r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_v10_2015_2016_stack.tif",
    r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_tp_2015_2016_stack.tif",
]

for f in era5_files:
    with rasterio.open(f) as src:
        print(f"\n📂 File: {f}")
        print(f"   Bands: {src.count}")
        print(f"   Shape: {src.height} x {src.width}")

        # Check for nodata value
        nodata = src.nodata
        print(f"   NoData metadata: {nodata}")

        # Read a sample band (1st, middle, last)
        for b in [1, src.count//2, src.count]:
            arr = src.read(b, masked=True)
            missing = np.sum(arr.mask) if hasattr(arr, "mask") else np.isnan(arr).sum()
            print(f"   Band {b}: missing pixels = {missing}")



📂 File: C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_t2m_2015_2016_stack.tif
   Bands: 17544
   Shape: 13 x 17
   NoData metadata: None
   Band 1: missing pixels = 0
   Band 8772: missing pixels = 0
   Band 17544: missing pixels = 0

📂 File: C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_d2m_2015_2016_stack.tif
   Bands: 17544
   Shape: 13 x 17
   NoData metadata: None
   Band 1: missing pixels = 0
   Band 8772: missing pixels = 0
   Band 17544: missing pixels = 0

📂 File: C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_u10_2015_2016_stack.tif
   Bands: 17544
   Shape: 13 x 17
   NoData metadata: None
   Band 1: missing pixels = 0
   Band 8772: missing pixels = 0
   Band 17544: missing pixels = 0

📂 File: C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_v10_2015_2016_stack.tif
   Bands: 17544
   Shape: 13 x 17
   NoData metadata: None
   Band 1: missing pixels = 0
   Band 8772: missing pixels = 0
   Band 17544: missing pixels 

In [1]:
import rasterio

ERA5_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_30m_stack_filled_final.tif"
VIIRS_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_label_2015_2016_30m_filled.tif"

with rasterio.open(ERA5_FILE) as src:
    print("ERA5 bands:", src.count)

with rasterio.open(VIIRS_FILE) as src:
    print("VIIRS bands:", src.count)


ERA5 bands: 5
VIIRS bands: 1


In [1]:
import rasterio
import glob
import os

# Path to your ERA5 folder
era5_dir = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_stacks_30m"
# Loop through all ERA5 tif files
for tif_file in glob.glob(os.path.join(era5_dir, "*.tif")):
    with rasterio.open(tif_file) as src:
        print(f"{os.path.basename(tif_file)} → {src.count} bands, shape: ({src.height}, {src.width})")


ERA5_d2m_resampled_30m.tif → 1 bands, shape: (11904, 13604)
ERA5_t2m_resampled_30m.tif → 1 bands, shape: (11904, 13604)
ERA5_tp_resampled_30m.tif → 1 bands, shape: (11904, 13604)
ERA5_u10_resampled_30m.tif → 1 bands, shape: (11904, 13604)
ERA5_v10_resampled_30m.tif → 1 bands, shape: (11904, 13604)
